# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc7ab6f6a90>
├── 'a' --> tensor([[-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940]])
└── 'x' --> <FastTreeValue 0x7fc7ab749a30>
    └── 'c' --> tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                        [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                        [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]])

In [4]:
t.a

tensor([[-0.4584,  0.1312, -0.1269],
        [ 1.3129, -0.0119, -0.6940]])

In [5]:
%timeit t.a

72.8 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc7ab6f6a90>
├── 'a' --> tensor([[ 1.6013,  1.0788,  1.2881],
│                   [ 0.3244,  0.5908, -0.1456]])
└── 'x' --> <FastTreeValue 0x7fc7ab749a30>
    └── 'c' --> tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                        [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                        [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]])

In [7]:
%timeit t.a = new_value

80.3 ns ± 1.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4584,  0.1312, -0.1269],
               [ 1.3129, -0.0119, -0.6940]]),
    x: Batch(
           c: tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]]),
       ),
)

In [10]:
b.a

tensor([[-0.4584,  0.1312, -0.1269],
        [ 1.3129, -0.0119, -0.6940]])

In [11]:
%timeit b.a

76 ns ± 0.709 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1726, -0.3270,  0.8072],
               [ 0.0541, -1.7466,  1.1799]]),
    x: Batch(
           c: tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]]),
       ),
)

In [13]:
%timeit b.a = new_value

654 ns ± 17.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.8 µs ± 475 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

301 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

278 µs ± 9.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc6f192f070>
├── 'a' --> tensor([[[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]],
│           
│                   [[-0.4584,  0.1312, -0.1269],
│                    [ 1.3129, -0.0119, -0.6940]]])
└── 'x' --> <FastTreeValue 0x7fc6f1f77f70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.5 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc6f191fc40>
├── 'a' --> tensor([[-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940],
│                   [-0.4584,  0.1312, -0.1269],
│                   [ 1.3129, -0.0119, -0.6940]])
└── 'x' --> <FastTreeValue 0x7fc6f8573910>
    └── 'c' --> tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                        [ 8.4956e-01,  1.6812e+00, -3.4257e-01,

In [23]:
%timeit t_cat(trees)

43.8 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.7 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                       [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                       [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]],
              
                      [[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                       [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                       [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]],
              
                      [[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                       [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                       [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]],
              
                      [[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                       [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                       [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00]],
      

In [26]:
%timeit Batch.stack(batches)

151 µs ± 3.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00],
                      [ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00],
                      [ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00],
                      [ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.1852e-01],
                      [ 8.4956e-01,  1.6812e+00, -3.4257e-01, -6.3874e-01],
                      [-9.4790e-01, -1.0118e-03, -1.3947e-02,  1.0904e+00],
                      [ 2.8167e-01,  2.6515e-01, -5.4665e-01, -2.18

In [28]:
%timeit Batch.cat(batches)

186 µs ± 4.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

532 µs ± 7.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
